# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 59 new papers today
          27 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/26 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.07874


extracting tarball to tmp_2302.07874... done.
Retrieving document from  https://arxiv.org/e-print/2302.07878


extracting tarball to tmp_2302.07878...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.07880


...

extracting tarball to tmp_2302.08386...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.08402


extracting tarball to tmp_2302.08402... done.
Retrieving document from  https://arxiv.org/e-print/2302.08405


extracting tarball to tmp_2302.08405...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07880-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07880) | **A red giant orbiting a black hole**  |
|| <mark>Kareem El-Badry</mark>, et al. -- incl., <mark>Hans-Walter Rix</mark>, <mark>Eleonora Zari</mark>, <mark>Melissa Hobson</mark>, <mark>Rhys Seeburger</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| *22 pages, 15 figures. Submitted to MNRAS*|
|**Abstract**| We report spectroscopic and photometric follow-up of a dormant black hole (BH) candidate from Gaia DR3. We show that the system, which we call Gaia BH2, contains a $\sim 1M_{\odot}$ red giant and a dark companion with mass $M_2 = 8.9\pm 0.3\,M_{\odot}$ that is very likely a BH. The orbital period, $P_{\rm orb} = 1277$ days, is much longer than that of any previously studied BH binary. Our radial velocity (RV) follow-up over a 6-month period spans most of the orbit's dynamic range in RV and is in excellent agreement with predictions of the Gaia solution. UV imaging and high-resolution optical spectra rule out all plausible luminous companions that could explain the orbit. The star is a bright ($G=12.3$), slightly metal-poor ($\rm [Fe/H]=-0.22$) low-luminosity giant ($T_{\rm eff}=4600\,\rm K$; $R = 7.9\,R_{\odot}$; $\log\left[g/\left({\rm cm\,s^{-2}}\right)\right] = 2.6$). The binary's orbit is moderately eccentric ($e=0.52$). The giant is strongly enhanced in $\alpha-$elements, with $\rm [\alpha/Fe] = +0.26$, but the system's Galactocentric orbit is typical of the thin disk. We obtained X-ray and radio nondetections of the source near periastron, which support BH accretion models in which the net accretion rate at the horizon is much lower than the Bondi-Hoyle-Lyttleton rate. At a distance of 1.16 kpc, Gaia BH2 is the second-nearest known BH, after Gaia BH1. Its orbit -- like that of Gaia BH1 -- seems too wide to have formed through common envelope evolution. Gaia BH1 and BH2 have orbital periods at opposite edges of the Gaia DR3 sensitivity curve, perhaps hinting at a bimodal intrinsic period distribution for wide BH binaries. Dormant BH binaries like Gaia BH1 and Gaia BH2 likely significantly outnumber their close, X-ray bright cousins, but their formation pathways remain uncertain. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07916-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07916) | **The CARMENES search for exoplanets around M dwarfs. Line-by-line  sensitivity to activity in M dwarfs**  |
|| M. Lafarga, et al. -- incl., <mark>Th. Henning</mark>, <mark>M. Kürster</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Radial velocities (RVs) measured from high-resolution stellar spectra are routinely used to detect and characterise orbiting exoplanet companions. The different lines present in stellar spectra are created by several species, which are non-uniformly affected by stellar variability features such as spots or faculae. Stellar variability distorts the shape of the spectral absorption lines from which precise RVs are measured, posing one of the main problems in the study of exoplanets. In this work we aim to study how the spectral lines present in M dwarfs are independently impacted by stellar activity. We used CARMENES optical spectra of six active early- and mid-type M dwarfs to compute line-by-line RVs and study their correlation with several well-studied proxies of stellar activity. We are able to classify spectral lines based on their sensitivity to activity in five M dwarfs displaying high levels of stellar activity. We further used this line classification to compute RVs with activity-sensitive lines and less sensitive lines, enhancing or mitigating stellar activity effects in the RV time series. For specific sets of the least activity-sensitive lines, the RV scatter decreases by ~ 2 to 5 times the initial one, depending on the star. Finally, we compare these lines in the different stars analysed, finding the sensitivity to activity to vary from star to star. Despite the high density of lines and blends present in M dwarf stellar spectra, we find that a line-by-line approach is able to deliver precise RVs. Line-by-line RVs are also sensitive to stellar activity effects, and they allow for an accurate selection of activity-insensitive lines to mitigate activity effects in RV. However, we find stellar activity effects to vary in the same insensitive lines from star to star. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07882-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07882) | **The role of baryons in self-interacting dark matter mergers**  |
|| <mark>Moritz S. Fischer</mark>, et al. -- incl., <mark>Nils-Henrik Durke</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| *14 pages, 10 figures + appendices, submitted to MNRAS*|
|**Abstract**| Mergers of galaxy clusters are promising probes of dark matter (DM) physics. For example, an offset between the DM component and the galaxy distribution can constrain DM self-interactions. We investigate the role of the intracluster medium (ICM) and its influence on DM-galaxy offsets in self-interacting dark matter (SIDM) models. To this end, we employ Smoothed Particle Hydrodynamics + N-body simulations to study idealised setups of equal- and unequal-mass mergers with head-on collisions. Our simulations show that the ICM hardly affects the offsets arising shortly after the first pericentre passage compared to DM-only (DMO) simulations. But later on, e.g. at the first apocentre, the offsets can be amplified by the presence of the ICM. Furthermore, we find that cross-sections small enough not to be excluded by measurements of the core sizes of relaxed galaxy clusters have a chance to produce observable offsets. We found that different DM models affect the DM distribution and also the galaxy and and ICM distribution, including its temperature. Potentially, the position of the shock fronts, combined with the brightest cluster galaxies (BCGs), provides further clues to the properties of DM. Overall our results demonstrate that mergers of galaxy clusters at stages about the first apocentre passage could be more interesting in terms of DM physics than those shortly after the first pericentre passage. This may motivate further studies of mergers at later evolutionary stages. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08386-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08386) | **Revealing the Dynamic Magneto-ionic Environments of Repeating Fast Radio  Burst Sources through Multi-year Polarimetric Monitoring with CHIME/FRB**  |
|| R. Mckinven, et al. -- incl., <mark>D. Z. Li</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| **|
|**Abstract**| Fast radio bursts (FRBs) display a confounding variety of burst properties and host galaxy associations. Repeating FRBs offer insight into the FRB population by enabling spectral, temporal and polarimetric properties to be tracked over time. Here, we report on the polarized observations of 12 repeating sources using multi-year monitoring with the Canadian Hydrogen Intensity Mapping Experiment (CHIME) over 400-800 MHz. We observe significant RM variations from many sources in our sample, including RM changes of several hundred $\rm{rad\, m^{-2}}$ over month timescales from FRBs 20181119A, 20190303A and 20190417A, and more modest RM variability ($\rm{\Delta RM \lesssim}$ few tens rad m$^{-2}$) from FRBs 20181030A, 20190208A, 20190213B and 20190117A over equivalent timescales. Several repeaters display a frequency dependent degree of linear polarization that is consistent with depolarization via scattering. Combining our measurements of RM variations with equivalent constraints on DM variability, we estimate the average line-of-sight magnetic field strength in the local environment of each repeater. In general, repeating FRBs display RM variations that are more prevalent/extreme than those seen from radio pulsars in the Milky Way and the Magellanic Clouds, suggesting repeating FRBs and pulsars occupy distinct magneto-ionic environments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08402-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08402) | **Continuum reverberation mapping of MCG 08-11-011**  |
|| C. Fian, et al. -- incl., <mark>T. Lewis</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| *arXiv admin note: text overlap with arXiv:2111.07385*|
|**Abstract**| We report the results from a photometric reverberation mapping campaign carried out with the C18 telescope at the Wise Observatory from 2019 to 2020, targeting the active galactic nucleus (AGN) MCG 08-11-011. The monitoring was conducted on a daily basis with specially designed narrow-band filters, spanning from optical to near-infrared wavelengths ($\sim4000$ to $8000${\AA}) and avoiding prominent broad emission lines. We aim to measure inter-band continuum time lags, determine the size-wavelength relation, and estimate the host-subtracted AGN luminosity for this system. We used the point-spread function photometry to extract the continuum light curves and measure the inter-band time lags using several methods, including the interpolated cross-correlation function, the z-transformed discrete correlation function, a von Neumann estimator, JAVELIN (in spectroscopic and photometric mode), MICA, and a multivariate correlation function. We find wavelength-dependent lags, $\tau(\lambda)$, up to $\sim$7 days between the multiband light curves of MCG 08-11-011. The observed lags are larger than predictions based on standard thin-disk theory by a factor of $\sim3-7$. We discern a significantly steeper ($\tau \propto \lambda^{4.74}$) size-wavelength relation than the $\tau \propto \lambda^{4/3}$ expected for a geometrically thin and optically thick accretion disk, which may result from the contribution of diffuse continuum emission to the flux. These results are similar to those found by previous continuum reverberation mapping campaigns. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08405-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08405) | **The X-ray invisible Universe. A look into the halos undetected by  eROSITA**  |
|| P. Popesso, et al. -- incl., <mark>A. Liu</mark>, <mark>J. Liske</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| *15 pages, 13 figures, Submitted to MNRAS*|
|**Abstract**| The paper presents the analysis of GAMA spectroscopic groups and clusters detected and undetected in the SRG/eROSITA X-ray map of the eFEDS (eROSITA Final Equatorial Depth Survey) area, in the halo mass range $10^{13}-5x10^{14}$ $M_{\odot}$ and at $z < 0.2$. We compare the X-ray surface brightness profiles of the eROSITA detected groups with the mean stacked profile of the undetected low-mass halos. Overall, we find that the undetected groups exhibit less concentrated X-ray surface brightness, dark matter, and galaxy distributions with respect to the X-ray detected halos. Consistently with the low mass concentration, the magnitude gap indicates that these are younger systems. The later assembly time is confirmed by the bluer average color of the BCG and of the galaxy population with respect to the detected systems. They reside with a higher probability in filaments while X-ray detected low-mass halos favor the nodes of the Cosmic Web. Because of the suppressed X-ray central emission, the undetected systems tend to be X-ray under-luminous at fixed halo mass, and to lie below the $L_X-M_{halo}$ relation. Interestingly, the X-ray detected systems inhabiting the nodes scatter the less around the relation, while those in filaments tend to lie below it. We do not observe any strong relation between the properties of detected and undetected systems with the AGN activity. The fraction of optically selected AGN in the galaxy population is consistent in the two samples. More interestingly, the probability that the BCG hosts a radio AGN is lower in the undetected groups. We, thus, argue that the observed differences between X-ray detected and undetected groups are ascribable to the Cosmic Web, and its role in the halo assembly bias. Our results suggest that the X-ray selection is biased to favor the most concentrated and old systems located in the nodes of the Cosmic Web. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07954-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07954) | **ATLAS-TEIDE: The next generations of ATLAS units for the Teide  Observatory**  |
|| <mark>Javier Licandro</mark>, et al. |
|*Appeared on*| *2023-02-17*|
|*Comments*| *5 pages, 8 figures, Conference paper*|
|**Abstract**| In this work we present the design of the ATLAS unit (Asteroid Terrestrial-impact Last Alert System) that will be installed at Teide Observatory in Tenerife island (Spain). ATLAS-Teide will be built by the Instituto de Astrofisica de Canarias (IAC) and will be operated as part of the ATLAS network in the framework of an operation and science exploitation agreement between the IAC and the ATLAS team at University of Hawaii. ATLAS-Teide will be the first ATLAS unit based on commercial on the shelf (COTS) components. Its design is modular, each module (building block) consist of four Celestron RASA 11 telescopes that point to the same sky field, equipped with QHY600PRO CMOS cameras on an equatorial Direct Drive mount. Each module is equivalent to a 56cm effective diameter telescope and provides a 7.3 deg^2 field of view and a 1.26 arcsec/pix plate scale. ATLAS-Teide will consist of four ATLAS modules in a roll-off roof building. This configuration allows to cover the same sky area of the actual ATLAS telescopes. The first ATLAS module was installed in November 2022 in an existing clamshell at the TO. This module (ATLAS-P) is being used as a prototype to test the system capabilities, develop the needed software (control, image processing, etc.) and complete the fully integration of ATLAS-Teide in the ATLAS network. The preliminary results of the tests are presented here, and the benefits of the new ATLAS design are discussed. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.08072-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.08072) | **Polarized accretion shocks from the cosmic web**  |
|| Tessa Vernstrom, et al. -- incl., <mark>George Heald</mark> |
|*Appeared on*| *2023-02-17*|
|*Comments*| *Published in Science Advances. Supplementary Materials available at this https URL*|
|**Abstract**| On the largest scales, galaxies are pulled together by gravity to form clusters, which are connected by filaments making a web-like pattern. Radio emission is predicted from this cosmic web, which should originate from the strong accretion shocks around the cosmic structures. We present the first observational evidence that Fermi-type acceleration from strong shocks surrounding the filaments of the cosmic web, as well as in peripherals of low-mass clusters, is at work in the Universe. Using all-sky radio maps and stacking on clusters and filaments, we have detected the polarization signature of the synchrotron emission with polarization fractions >= 20%, which is best explained by the organization of local magnetic fields by strong shock waves both at the cluster peripheries and between clusters. Our interpretation is well supported by a detailed comparison with state-of-the-art cosmological simulations. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.07880.md
    + _build/html/tmp_2302.07880/./gaia_bh2_population.png
    + _build/html/tmp_2302.07880/./gaia_bh2_porb_distribution.png
    + _build/html/tmp_2302.07880/./gaia_bh2_gost.png
exported in  _build/html/2302.07916.md
    + _build/html/tmp_2302.07916/./figures/J07446+035/correlation_ccfrvnzp_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png
    + _build/html/tmp_2302.07916/./figures/J07446+035/correlation_ccfbis_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png
    + _build/html/tmp_2302.07916/./figures/J07446+035/correlation_serval

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\kb}[1]{[\color{teal}{KB: #1}]}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\kb$}[1]{[\color{teal}{KB: #1}]}$
$\newcommand{$\red$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\}{mn}$
$\newcommand{\@}{tempa}$
$\newcommand{\@}{tempa }$
$\newcommand{\@}{tempb }$
$\newcommand{\@}{tempc$
$  }$
$\newcommand{\@}{tempb }$</div>



<div id="title">

# A red giant orbiting a black hole

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07880-b31b1b.svg)](https://arxiv.org/abs/2302.07880)<mark>Appeared on: 2023-02-17</mark> - _22 pages, 15 figures. Submitted to MNRAS_

</div>
<div id="authors">

<mark><mark>Kareem El-Badry</mark></mark>, et al. -- incl., <mark><mark>Hans-Walter Rix</mark></mark>, <mark><mark>Eleonora Zari</mark></mark>, <mark><mark>Melissa Hobson</mark></mark>, <mark><mark>Rhys Seeburger</mark></mark>

</div>
<div id="abstract">

**Abstract:** We report spectroscopic and photometric follow-up of a dormant black hole (BH) candidate from{\it Gaia}DR3. We show that the system, which we call Gaia BH2, contains a$\sim$1$M_{\odot}$red giant and a dark companion with mass$M_2 = 8.9\pm 0.3 M_{\odot}$that is very likely a BH. The orbital period,$P_{\rm orb} = 1277$days, is much longer than that of any previously studied BH binary. Our radial velocity (RV) follow-up over a 6-month period spans most of the orbit's dynamic range in RV and is in excellent agreement with predictions of the{\it Gaia}solution. UV imaging and high-resolution optical spectra rule out all plausible luminous companions that could explain the orbit. The  star is a bright ($G=12.3$), slightly metal-poor ($\rm[Fe/H]=-0.22$) low-luminosity giant ($T_{\rm eff}=4600 \rm K$;$R = 7.9 R_{\odot}$;$\log\left[g/\left({\rm cm s^{-2}}\right)\right] = 2.6$). The binary's orbit is moderately eccentric ($e=0.52$). The giant is strongly enhanced in$\alpha-$elements, with$\rm[\alpha/Fe] = +0.26$, but the system's Galactocentric orbit is typical of the thin disk. We obtained X-ray and radio nondetections of the source near periastron, which support BH accretion models in which the net accretion rate at the horizon is much lower than the Bondi-Hoyle-Lyttleton rate. At a distance of 1.16 kpc, Gaia BH2 is the second-nearest known BH, after Gaia BH1.  Its orbit -- like that of Gaia BH1 -- seems too wide to have formed through common envelope evolution. Gaia BH1 and BH2 have orbital periods at opposite edges of the{\it Gaia}DR3 sensitivity curve, perhaps hinting at a bimodal intrinsic period distribution for wide BH binaries. Dormant BH binaries like Gaia BH1 and Gaia BH2 likely significantly outnumber their close, X-ray bright cousins, but their formation pathways remain uncertain.

</div>

<div id="div_fig1">

<img src="tmp_2302.07880/./gaia_bh2_population.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison of Gaia BH1 and BH2 (black points) to other known BHs. Red and blue symbols correspond to accreting BHs with low- and high-mass companions. Magenta symbols show detached binaries in the globular cluster NGC 3201, and cyan points show detached binaries in which the luminous star is a high-mass ($\gtrsim 20 M_{\odot}$) star. Gaia BH2 stands out from the rest of the population most strongly due to its orbital period, which is $7\times$ longer than that of Gaia BH1 and $\sim 1000$ times longer than the periods of typical X-ray binaries. (*fig:bh_pop*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07880/./gaia_bh2_porb_distribution.png" alt="Fig2" width="100%"/>

**Figure 2. -** Orbital period distribution of all astrometric binaries in {\it Gaia} DR3 (black) and Gaia BH1 and BH2 (red). In the bottom panel, we divide the observed period distribution of all astrometric binaries by the expected intrinsic binary period distribution for solar-type stars. This provides an approximate sensitivity function of {\it Gaia} DR3 as a function of orbital period, marginalized over other observables. The two BHs have periods at the long- and short-period edges of the full observed period distribution, where DR3's sensitivity is $\sim 10\times$ lower than at its peak at 1.5 years. This is suggestive of a bimodal intrinsic period distribution, with fewer BHs at periods of 1-3 years. (*fig:porb_distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07880/./gaia_bh2_gost.png" alt="Fig3" width="100%"/>

**Figure 3. -** Predicted observation times of Gaia BH2 from the {\it Gaia} observation scheduling tool (GOST). Black line shows the best-fit orbit from our combined fit. Red points show the predicted photocenter positions at the times when GOST predicts {\it Gaia} would have observed the source. Note that we do not have access to the actual measured $\Delta \rm RA$, $\Delta \rm Dec$ and $\rm RV$ values; only to the predicted scan times. Further caveats are discussed in the text. (*fig:gost*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\ccf}{cross-correlation function\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}\xspace}$
$\newcommand{\ms}{\ensuremath{\mathrm{m s^{-1}}}\xspace}$
$\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}\xspace}$
$\newcommand{\A}{\ensuremath{\mathrm{\AA}}\xspace}$
$\newcommand{\Halpha}{\ensuremath{\mathrm{H\alpha}}\xspace}$
$\newcommand{\CaHK}{\ion{Ca}{II} H\&K\xspace}$
$\newcommand{\pEWHalpha}{pEW\ensuremath{'(\Halpha)}\xspace}$
$\newcommand{\IHalpha}{\ensuremath{I_{\Halpha}}\xspace}$
$\newcommand{\logLHalphaLbol}{\ensuremath{\log(L_{\Halpha}/L_{\mathrm{bol}})}\xspace}$
$\newcommand{\LXLJ}{\ensuremath{L_X/L_J}\xspace}$
$\newcommand{\RHK}{\ensuremath{R'_{\mathrm{HK}}}\xspace}$
$\newcommand{\logRHK}{\ensuremath{\log R'_{\mathrm{HK}}}\xspace}$
$\newcommand{\HeD}{\ion{He}{I} \ensuremath{\mathrm{D_3}}\xspace}$
$\newcommand{\Heten}{\ensuremath{\mathrm{\ion{He}{I}} \lambda10833 \mathrm{\AA}}\xspace}$
$\newcommand{\Pabeta}{\ensuremath{\mathrm{Pa}\beta}\xspace}$
$\newcommand{\NaD}{\ensuremath{\mathrm{\ion{Na}{I} D}}\xspace}$
$\newcommand{\Caii}{\ensuremath{\mathrm{\ion{Ca}{II}}}\xspace}$
$\newcommand{\Prot}{\ensuremath{P_{\mathrm{rot}}}\xspace}$
$\newcommand{\Prothalf}{\ensuremath{\frac{1}{2}P_{\mathrm{rot}}}\xspace}$
$\newcommand{\Protthird}{\ensuremath{\frac{1}{3}P_{\mathrm{rot}}}\xspace}$
$\newcommand{\Protalias}{\ensuremath{P_{\mathrm{rot}}\mathrm{ 1 d a.}}\xspace}$
$\newcommand{\Prothalfalias}{\ensuremath{\frac{1}{2}P_{\mathrm{rot}}\mathrm{ 1 d a.}}\xspace}$
$\newcommand{\Protthirdalias}{\ensuremath{\frac{1}{3}P_{\mathrm{rot}}\mathrm{ 1 d a.}}\xspace}$
$\newcommand{\days}{\ensuremath{\mathrm{d}}\xspace}$
$\newcommand{\FAP}{\ensuremath{\mathrm{FAP}}\xspace}$
$\newcommand{\Msun}{\ensuremath{\mathrm{M_{\odot}}}\xspace}$
$\newcommand{\Mearth}{\ensuremath{\mathrm{M_{\oplus}}}\xspace}$
$\newcommand{\Mjup}{\ensuremath{\mathrm{M_{J}}}\xspace}$
$\newcommand{\caracal}{\texttt{caracal}\xspace}$
$\newcommand{\serval}{\texttt{serval}\xspace}$
$\newcommand{\raccoon}{\texttt{raccoon}\xspace}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\ccf$}{cross-correlation function$\xspace$}$
$\newcommand{$\vsini$}{$\ensuremath${v \sin{i}}$\xspace$}$
$\newcommand{$\ms$}{$\ensuremath${\mathrm{m s^{-1}}}$\xspace$}$
$\newcommand{$\kms$}{$\ensuremath${\mathrm{km s^{-1}}}$\xspace$}$
$\newcommand{$\A$}{$\ensuremath${\mathrm{$\A$A}}$\xspace$}$
$\newcommand{$\Halpha$}{$\ensuremath${\mathrm{H\alpha}}$\xspace$}$
$\newcommand{$\CaHK$}{$\ion${Ca}{II} H\&K$\xspace$}$
$\newcommand{$\pEWHalpha$}{pEW$\ensuremath${'($\Halpha$)}$\xspace$}$
$\newcommand{$\IHalpha$}{$\ensuremath${I_{$\Halpha$}}$\xspace$}$
$\newcommand{$\logLHalphaLbol$}{$\ensuremath${\log(L_{$\Halpha$}/L_{\mathrm{bol}})}$\xspace$}$
$\newcommand{$\LXLJ$}{$\ensuremath${L_X/L_J}$\xspace$}$
$\newcommand{$\RHK$}{$\ensuremath${R'_{\mathrm{HK}}}$\xspace$}$
$\newcommand{$\logRHK$}{$\ensuremath${\log R'_{\mathrm{HK}}}$\xspace$}$
$\newcommand{$\HeD$}{$\ion${He}{I} $\ensuremath${\mathrm{D_3}}$\xspace$}$
$\newcommand{$\Heten$}{$\ensuremath${\mathrm{$\ion${He}{I}} \lambda10833 \mathrm{$\A$A}}$\xspace$}$
$\newcommand{$\Pabeta$}{$\ensuremath${\mathrm{Pa}\beta}$\xspace$}$
$\newcommand{$\NaD$}{$\ensuremath${\mathrm{$\ion${Na}{I} D}}$\xspace$}$
$\newcommand{$\Caii$}{$\ensuremath${\mathrm{$\ion${Ca}{II}}}$\xspace$}$
$\newcommand{$\Prot$}{$\ensuremath${P_{\mathrm{rot}}}$\xspace$}$
$\newcommand{$\Prot$half}{$\ensuremath${\frac{1}{2}P_{\mathrm{rot}}}$\xspace$}$
$\newcommand{$\Prot$third}{$\ensuremath${\frac{1}{3}P_{\mathrm{rot}}}$\xspace$}$
$\newcommand{$\Prot$alias}{$\ensuremath${P_{\mathrm{rot}}\mathrm{ 1 d a.}}$\xspace$}$
$\newcommand{$\Prot$halfalias}{$\ensuremath${\frac{1}{2}P_{\mathrm{rot}}\mathrm{ 1 d a.}}$\xspace$}$
$\newcommand{$\Prot$thirdalias}{$\ensuremath${\frac{1}{3}P_{\mathrm{rot}}\mathrm{ 1 d a.}}$\xspace$}$
$\newcommand{$\days$}{$\ensuremath${\mathrm{d}}$\xspace$}$
$\newcommand{$\FAP$}{$\ensuremath${\mathrm{FAP}}$\xspace$}$
$\newcommand{$\Msun$}{$\ensuremath${\mathrm{M_{\odot}}}$\xspace$}$
$\newcommand{$\Mearth$}{$\ensuremath${\mathrm{M_{\oplus}}}$\xspace$}$
$\newcommand{$\Mjup$}{$\ensuremath${\mathrm{M_{J}}}$\xspace$}$
$\newcommand{$\caracal$}{\texttt{caracal}$\xspace$}$
$\newcommand{$\serval$}{\texttt{serval}$\xspace$}$
$\newcommand{$\raccoon$}{\texttt{raccoon}$\xspace$}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07916-b31b1b.svg)](https://arxiv.org/abs/2302.07916)<mark>Appeared on: 2023-02-17</mark> - _Accepted for publication in A&A_

</div>
<div id="authors">

M. Lafarga, et al. -- incl., <mark><mark>Th. Henning</mark></mark>, <mark><mark>M. Kürster</mark></mark>

</div>
<div id="abstract">

**Abstract:** Radial velocities (RVs) measured from high-resolution stellar spectra are routinely used to detect and characterise orbiting exoplanet companions. The different lines present in stellar spectra are created by several species, which are non-uniformly affected by stellar variability features such as spots or faculae. Stellar variability distorts the shape of the spectral absorption lines from which precise RVs  are measured, posing one of the main problems in the study of exoplanets.In this work we aim to study how the spectral lines present in M dwarfs are independently impacted by stellar activity.We used CARMENES optical spectra of six active early- and mid-type M dwarfs to compute line-by-line RVs and study their correlation with several well-studied proxies of stellar activity.We are able to classify spectral lines based on their sensitivity to activity in five M dwarfs displaying high levels of stellar activity. We further used this line classification to compute RVs with activity-sensitive lines and less sensitive lines, enhancing or mitigating stellar activity effects in the RV time series. For specific sets of the least activity-sensitive lines, the RV scatter decreases by$\sim2$to 5 times the initial one, depending on the star. Finally, we compare these lines in the different stars analysed, finding the sensitivity to activity to vary from star to star.Despite the high density of lines and blends present in M dwarf stellar spectra, we find that a line-by-line approach is able to deliver precise RVs. Line-by-line RVs are also sensitive to stellar activity effects, and they allow for an accurate selection of activity-insensitive lines to mitigate activity effects in RV. However, we find stellar activity effects to vary in the same insensitive lines from star to star.

</div>

<div id="div_fig1">

<img src="tmp_2302.07916/./figures/J07446+035/correlation_ccfrvnzp_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png" alt="Fig7" width="100%"/>

**Figure 7. -** Line-by-line results obtained using inactive lines for J07446+035 (YZ CMi). \emph{Top left}: LAV RV time series obtained using RVs of selected inactive individual lines (colour), together with the LAV RV time series obtained using all the lines (black). The lines are selected based on the correlation of their RVs with the total spectrum RV, and their scatter. \emph{Middle left}: Number of lines used to compute the average RV in each observation, for the different selection of lines used in the \emph{top left panel}. \emph{Top right}: Distribution of the Pearson's correlation coefficient R from the correlation between the individual line RVs and the total spectrum RV. The regions in colour indicate the different selection of lines used to compute the RVs of the \emph{top left panel}. Additionally, we show in grey the distribution of R values after applying only the cut in line RV scatter (wstd). \emph{Bottom left}: Periodograms of the RV datasets plotted on the \emph{top left panel}. The horizontal lines correspond to the 0.1, 0.01, and 0.001 FAP levels, respectively. \emph{Bottom right}: Zoom in of the periodogram region around the peak corresponding to the rotation period of the star. The text indicates the period and power of the highest peak of each periodogram. (*fig:tsnhpccfrvinactivebestJ07446+035*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07916/./figures/J07446+035/correlation_ccfbis_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png" alt="Fig19" width="100%"/>

**Figure 19. -** Same as Fig. \ref{fig:tsnhpccfrvinactivebestJ07446+035}, but using R values from the correlation between the individual line RVs and CCF BIS. (*fig:tsnhpccfbisinactivebestJ07446+035*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07916/./figures/J07446+035/correlation_servalcrx_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png" alt="Fig20" width="100%"/>

**Figure 20. -** Same as Fig. \ref{fig:tsnhpccfrvinactivebestJ07446+035}, but using R values from the correlation between the individual line RVs and CRX. (*fig:tsnhpservalcrxinactivebestJ07446+035*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

208  publications in the last 7 days.
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers